# Politician Activity on Wikipedia

The parameters in the cell below can be adjusted to explore otherPoliticians and time frames.

### How to explore other politicians?
The ***politician_ID*** is an internal identifier that connects the different social media accounts. You can [use this other notebook](../Politicians.ipynb?autorun=true) to get other the identifiers of otherPoliticians.

***Alternatively***, you can direcly use the [Politicians API](http://mediamonitoring.gesis.org/api/Politicians/swagger/), or access it with the [SMM Wrapper](https://pypi.org/project/smm-wrapper/).

In [ ]:
# Parameters: 
politician_id = 240
aggregation = 'month' # aggregation can be by day ('day'), week ('week') or month('month')
from_date='2017-09-01'
to_date='2018-12-31'

## Using API

In [ ]:
from smm_wrapper import SMMPoliticians
from wikiwho_wrapper import WikiWho

# Create an instance to the smm wrapper
smm = SMMPoliticians()

# Request the politicians from the API
df = smm.dv.get_politicians()

In [ ]:
import pandas as pd
#using the api to get the data
wiki_chobs = smm.dv.wikipedia(_id=politician_id, from_date=from_date, to_date=to_date, aggregate_by=aggregation)
wiki_chobs = wiki_chobs.groupby('date').agg({'chobs': 'sum'}).reset_index()
wiki_ids = df.loc[df.index == politician_id, 'wp_ids'].iloc[0]

#using wikiwho to extract conflicts and revisions
ww = WikiWho(lng='de')
wiki_data = pd.concat(ww.dv.edit_persistence(page_id=wiki_id, start=from_date, end=to_date) for wiki_id in wiki_ids)
wiki_data['undos'] = wiki_data['dels'] + wiki_data['reins']
wiki_data = wiki_data[['year_month','conflict', 'elegibles', 'revisions', 'undos']]
wiki_data['date'] = pd.to_datetime(wiki_data['year_month'])
wiki_data = wiki_data.groupby('date').agg({'conflict': 'sum','elegibles':'sum', 'revisions':'sum','undos':'sum' }).reset_index()
wiki_data['conflict_score'] = wiki_data['conflict'] / wiki_data['elegibles']
wiki_data.fillna(0, inplace=True)
#using wikipedia and wikimedia to extract views
import wikipedia
import requests
wikipedia.set_lang('de')
session = requests.Session()
wiki_title = wikipedia.page(pageid=wiki_ids).title
url = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/user/{}/daily/{}/{}'.format(wiki_title,from_date.replace('-',''),to_date.replace('-',''))
views = pd.DataFrame(session.get(url=url).json()['items'])
views = views[['timestamp', 'views']]
views['timestamp']=pd.to_datetime(views['timestamp'], format='%Y%m%d%H')
if aggregation  == 'week':    
    views = views.groupby([pd.Grouper(key='timestamp', freq='W-SUN')])['views'].sum().reset_index().sort_values('timestamp')
    views['timestamp'] = views['timestamp'] - pd.Timedelta(days=6)
    
elif aggregation  == 'month':    
    views = views.groupby([pd.Grouper(key='timestamp', freq='MS')])['views'].sum().reset_index().sort_values('timestamp')


## Plotting

In [ ]:
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [go.Scatter(x=wiki_chobs['date'], y=wiki_chobs['chobs'], name='Changes', line_shape='spline'),
            go.Scatter(x=views['timestamp'], y=views['views'], name='Views', yaxis='y2')], 
    "layout": go.Layout( title='Wikipedia Activity', xaxis={'title':'Date'}, yaxis=dict(
        title='Views and Changes'
    ),
    yaxis2=dict(
        title='Views',
        
        overlaying='y',
        side='right'
    ), showlegend=True)
})
plotly.offline.iplot({
    "data": [go.Scatter(x=wiki_data['date'], y=wiki_data['undos'], name='Undos', line_shape='spline'),
            go.Scatter(x=wiki_data['date'], y=wiki_data['conflict_score'], name='Conflict', line_shape='spline', yaxis='y2')], 
    "layout": go.Layout(title='Wikipedia Disagreement', xaxis={'title':''}, yaxis=dict(
        title='Undos and Conflict'
    ),
    yaxis2=dict(
        title='Conflict',
        
        overlaying='y',
        side='right'
    ), showlegend=True)
})
